In [1]:
import pandas as pd

# Define the file path
file_path = 'netflix_titles.csv'

df = pd.read_csv(file_path)
print("First 5 records:", df.head())

First 5 records:   show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September

In [2]:
import os

# Define the path to the directory you want to list
# '.' refers to the current directory
path = '.'

print(os.listdir(path))

['.config', 'netflix_titles.csv', 'sample_data']


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def load_data(df):
    if 'type' not in df.columns:
        raise ValueError('CSV must contain a "type" column (Movie / TV Show)')
    for c in ['date_added','release_year','country','listed_in','duration','rating']:
        if c not in df.columns:
            df[c] = None
    df['primary_genre'] = df['listed_in'].fillna('').apply(lambda s: s.split(',')[0] if s else 'Unknown')
    def minutes_from_duration(x):
        try:
            if isinstance(x, str) and 'min' in x:
                return int(x.split()[0])
            else:
                return np.nan
        except:
            return np.nan
    df['minutes'] = df['duration'].apply(minutes_from_duration)
    return df

def create_pdf(df, out_pdf='netflix_eda_real.pdf'):
    type_counts = df['type'].value_counts()
    top_countries = df['country'].value_counts().head(8)
    year_counts = df['release_year'].value_counts().sort_index()
    top_genres = df['primary_genre'].value_counts().head(8)
    rating_counts = df['rating'].value_counts()

    with PdfPages(out_pdf) as pp:
        fig, axes = plt.subplots(3,1, figsize=(8.27,11.69))
        fig.suptitle("Netflix Movies & TV Shows — Exploratory Data Analysis (REAL DATA)", fontsize=14, y=0.95)
        axes[0].bar(type_counts.index, type_counts.values); axes[0].set_title("Content Type Distribution"); axes[0].set_ylabel("Count")
        axes[1].barh(top_genres.index[::-1], top_genres.values[::-1]); axes[1].set_title("Top Primary Genres"); axes[1].set_xlabel("Count")
        years_sorted = np.sort(year_counts.index); year_series = year_counts.reindex(years_sorted, fill_value=0)
        year_smoothed = year_series.rolling(window=3, center=True, min_periods=1).mean()
        axes[2].plot(years_sorted, year_smoothed); axes[2].set_title("Releases by Year (3-year rolling mean)"); axes[2].set_xlabel("Year"); axes[2].set_xlim(1980, 2025)
        plt.tight_layout(rect=[0, 0, 1, 0.93]); pp.savefig(fig); plt.close(fig)
        fig2, axes = plt.subplots(3,1, figsize=(8.27,11.69))
        fig2.suptitle("Further Insights - Countries, Duration & Ratings", fontsize=14, y=0.95)
        axes[0].bar(top_countries.index, top_countries.values); axes[0].set_title("Top Producing Countries"); axes[0].set_ylabel("Count"); axes[0].tick_params(axis='x', rotation=45)
        axes[1].hist(df['minutes'].dropna(), bins=20); axes[1].set_title("Movie Duration Distribution (minutes)"); axes[1].set_xlabel("Minutes")
        axes[2].bar(rating_counts.index, rating_counts.values); axes[2].set_title("Content Ratings Distribution"); axes[2].set_ylabel("Count")
        plt.tight_layout(rect=[0, 0, 1, 0.93]); pp.savefig(fig2); plt.close(fig2)
    print('PDF saved to: {}'.format(out_pdf))

processed_df = load_data(df.copy())
create_pdf(processed_df)

PDF saved to: netflix_eda_real.pdf
